In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from sklearn import *
# from PG.Software.MLService.ML_train import *
from favel.Software.MLService.ML import ML

In [2]:
from pathlib import Path

data=pd.read_csv('./test/bpdp_outputs.csv')

data=data.rename(columns={'true_value': 'truth'})

print(data.shape)
data.head()


(407, 14)


,subject,predicate,object,truth,COPAAL,degree_product,jaccard,Katz,KL,klrel,KS,pathent,simrank,adamic_adar
0,http://dbpedia.org/resource/Alexander_Kerensky,http://dbpedia.org/ontology/birthPlace,http://dbpedia.org/resource/Ulyanovsk,1,0.999998,1064,0.027397,0.027750,0.107504,0.099132,0.046729,3981.216920,0.000787,0.336008
1,http://dbpedia.org/resource/Ernst_Mayr,http://dbpedia.org/ontology/deathPlace,"http://dbpedia.org/resource/Bedford,_Massachus...",1,0.995696,2750,0.009615,0.019875,0.059052,0.061001,0.022926,3093.521205,0.000236,0.079083
2,http://dbpedia.org/resource/J._R._R._Tolkien,http://dbpedia.org/ontology/deathPlace,http://dbpedia.org/resource/Bournemouth,1,0.999985,38610,0.004494,0.103625,0.059549,0.078402,0.034786,3160.010453,0.000308,0.259814
3,http://dbpedia.org/resource/André_Weil,http://dbpedia.org/ontology/deathPlace,"http://dbpedia.org/resource/Princeton,_New_Jersey",1,0.851353,10032,0.000000,0.077375,0.076913,0.096255,0.068463,5778.382026,0.000600,0.000000
4,http://dbpedia.org/resource/John_A._Macdonald,http://dbpedia.org/ontology/deathPlace,http://dbpedia.org/resource/Ottawa,1,1.000000,127185,0.002335,0.864750,0.089868,0.116327,0.113633,2084.462668,0.001092,1.286593


In [3]:
data.truth.value_counts(normalize=False)

0    204
1    203
Name: truth, dtype: int64

# Train

In [4]:
path='./test/output_5'
Path(path).mkdir(parents=True, exist_ok=True)
ml = ML(log_file=f'{path}/test.logs')

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
train_result = ml.train_model(df=data,
                    ml_model=GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0), 
                    output_path=path,
                    dataset_path='BPDP')



In [ ]:
train_result

# Validate

In [ ]:
test_df=data.sample(100).copy()
print(test_df.shape)
test_df.head(1)
     

In [ ]:
test_df.ensemble_score.value_counts()

In [ ]:
testing_result = ml.validate_model(test_df, 
                                    output_path=path,
                                    dataset_path='BPDP')

testing_result

In [ ]:
testing_result.truth.value_counts()

# Test Optmisation

In [6]:
ml_model_name='GradientBoostingClassifier'

# ml_model_params = """[
#     {'name': 'n_estimators', 'range':[50, 150], 'prior':'log-uniform'},
#     {'name':'min_samples_leaf', 'range': [1.0,10.0]},
#     {'name':'max_features', 'range': ['sqrt', 'log2']},
# ]"""

ml_model_params = "{'n_estimators': 100, 'min_samples_leaf': 1.0, 'max_features': 1, 'random_state': 0}"

ml_model = ml.get_sklearn_model(ml_model_name, ml_model_params, data)
type(ml_model)


sklearn.ensemble._gb.GradientBoostingClassifier

In [7]:
model, lableEncoder, trainMetrics = ml.train_model(df=data, 
                                    ml_model=ml_model, 
                                    output_path=path, 
                                    dataset_path='BPDP')

TRAIN:  (407, 11) (407,) GradientBoostingClassifier(max_features=1, min_samples_leaf=1.0, random_state=0) int64


In [8]:
trainMetrics

{'overall': 0.5, 'cv_mean': 0.5, 'cv_std': 0.0}

# Optimisation

In [ ]:
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
dataframe = read_csv(url, header=None)
# split into input and output elements
data = dataframe.values
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)
# define search space
params = dict()
params['C'] = (1e-6, 100.0, 'log-uniform')
params['gamma'] = (1e-6, 100.0, 'log-uniform')
params['degree'] = (1,5)
params['kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']
# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define the search
search = BayesSearchCV(estimator=SVC(), search_spaces=params, n_jobs=-1, cv=cv)
# perform the search
search.fit(X, y)
# report the best result
print(search.best_score_)
print(search.best_params_)

# Sk-Opt

In [ ]:
X, y = data.drop(['subject','predicate','object','truth'], axis=1), data['truth']
X.shape, y.shape

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.75,
                                                    random_state=0)

# log-uniform: understand as search over p = exp(x) by varying x
opt = BayesSearchCV(
    RandomForestClassifier(),
    {
        'n_estimators': Integer(1, 50000, prior='log-uniform'),
        'min_samples_leaf': Real(1,10),
        'max_features': Categorical(['sqrt', 'log2'])
    },
    n_iter=5,
    random_state=0
)

# executes bayesian optimization
_ = opt.fit(X_train, y_train)

# model can be saved, used for predictions or scoring
print(opt.score(X_test, y_test))

In [ ]:
dict(opt.best_params_)

In [ ]:
realparams={
        'n_estimators': Integer(1, 50000, prior='log-uniform'),
        'min_samples_leaf': Real(1,10),
        'max_features': Categorical(['sqrt', 'log2'])
}

p = """[
    {'name': 'n_estimators', 'range':[50, 150], 'prior':'log-uniform'},
    {'name':'min_samples_leaf', 'range': [1.0,10.0]},
    {'name':'max_features', 'range': ['sqrt', 'log2']},
]"""
p

In [ ]:
p=ast.literal_eval(p)
p

In [ ]:
import ast
from skopt.space import Real, Categorical, Integer


result={}

# type(p)
for x in p:
    if type(x['range'])==tuple:
        if type(x['range'][0]) == int: 
            result[x['name']] = Integer(x['range'][0], x['range'][1])
        elif type(x['range'][0]) == float: 
            result[x['name']] = Real(x['range'][0], x['range'][1])
    elif type(x['range']) == list:
        result[x['name']] = Categorical(x['range'])

result